# Order Brushing Solution for Shopee Code League 2020

## The task is to find shopid's along with userid's that possibly have done order brushing 

## Brushing is a deceitful technique sometimes used in e-commerce to boost a seller's ratings by creating fake orders

## The participants were given a csv file containing transactions from a certain period of time in shopee

## The result is also saved in a csv file

## if the shopid didn't suspected to do order brushing, then the value in userid is 0

In [23]:
import timeit
import pandas as pd
import numpy as np
start = timeit.default_timer() #to calculate running time

In [30]:
#basically this is how you do it
#The Answer iS UNIXTIME
#convert the timestamp into unixtime, 
#then find the value difference between the transaction you want to know  their proximity,

#My answer is if a userid did atleast 3 transactions in the same shopid within 1 hour instance 
#And if the unixtime transaction between the first instance and the last instance is less than 3600 (1hour = 60minutes = 3600seconds)
#Then that shopid along with userid is suspected to have done order brushing

#the order is:
#add a new column (unix_time) by transforming event_time
#group by shopid and userid and count the numbers of transactions, then only take shopid that has more than 3 transactions to save time
# after that, sort the value based on unix_time from highest to lowest and  then group them by shopid, userid, and unixtime 
# Make a dictionary to split the dataframe based on shopid (keys)
# Then make another nested dict consists of shopid (keys) and the values are splitted dataframe of userid's in that shopid

#use this nested dict
#then make another nested dict for the answer
#make nested loops,  first loops through each shopid and then through each userid to get the dataframe 
# take the first time stamp, absolut substract with the last time stamps in the unix_time column 
# if the result is  within <3600 time period, 
# it means all of the transactions of that userid in that shopid is within 1 hour period (because we have sort the unixtime)
# save the value in the answer dictionary
# then break and move to the next shopid/userid 
# if not <3600, keep looping by taking the 1st unixtime value again, but this time absolut substract it with the second to the last timestamp
#if yes <3600 all the items is within 1 hour period,  save the value then break
#if still not, keep doing this until reach the 3rd value in the list (minimum 3 trans to be considered as order brushing), 
#if yes , save and break
#if not, it means this userid didnot do the order brushing in this shopid
#move to the next userid, and if done, our loops will move to the next shopid

In [25]:
df = pd.read_csv('order_brush_order.csv') #the dataframe
df #examine
#4 columns, 222750 rows
#df.isnull().values.any() returned false, means there aren't any null values

,orderid,shopid,userid,event_time
0,31076582227611,93950878,30530270,2019-12-27 00:23:03
1,31118059853484,156423439,46057927,2019-12-27 11:54:20
2,31123355095755,173699291,67341739,2019-12-27 13:22:35
3,31122059872723,63674025,149380322,2019-12-27 13:01:00
4,31117075665123,127249066,149493217,2019-12-27 11:37:55
...,...,...,...,...
222745,31245479170194,110868129,193218997,2019-12-28 23:17:59
222746,31230440360374,91639906,3541807,2019-12-28 19:07:20
222747,31191471145838,29391773,135908070,2019-12-28 08:17:52
222748,31198471732255,61556313,27306783,2019-12-28 10:14:31


In [26]:
#create PipeLine
def order_brush(df):
    time_zero = pd.to_datetime('1970-01-01') #set timezero for unixtime
    df["unix_time"] = pd.to_datetime(df["event_time"]).apply(lambda x: int((x - time_zero).total_seconds())) #change to datetime object then apply unix time transformation
    df_sorted = df.sort_values('unix_time') # Sort the rows of `df` ascendingly by `unit_time`. 
    
    # first phase, sorting and taking only shopid that has more than 3 transactions to save running time
    order = df_sorted.groupby(['shopid', 'userid'])[['userid']].count().rename(columns={'userid':'transaction_counts'})
    order = order.sort_values(by=['transaction_counts'], ascending=False).reset_index()
    order = order[order['transaction_counts'] >=3] #to save time, we are not going to check shopid that has less than 3 transaction, as our formula stated less than 3 not order brushing

    shopid = order['shopid'] #unique shopid values
    
    # then make a dataframe that's ready to be use
    df_use = df[df['shopid'].isin(shopid)].reset_index(drop=True)
    df_use = df_use[['shopid', 'userid', 'unix_time']].sort_values(by=['shopid', 'userid', 'unix_time'], ascending=True)
    
    
    #second Phase
    #make a dictionary consisting dataframes of each shopid as the keys
    #do this by spliting the dataframe based on each shopid

    df_shopid = {}

    shopid = df_use['shopid'].unique()
    userid = df_use['userid'].unique()

    for i in shopid:
        temp_df = df_use[df_use['shopid'] == i]
        df_shopid[i] = temp_df
    
    
    #now the Third Phase
    #make a nested dict for each shopid (keys) 
    #and the value for each shopid (keys) is also a dictionary consisting dataframes of each userid in that shopid

    df_shopid_complete = {}

    for i in df_shopid.keys():
        temp_dict = {}
        for x in df_shopid[i]['userid'].unique():
            temp_dict[x] = df_shopid[i][df_shopid[i]['userid'] == x]
        df_shopid_complete[i] = temp_dict
        
    #Fourth Phase
    #now time to find the answer
    #make nested dict again but for the answer
    # for each keys, the value is a dictionary with the userid as keys and number of transaction within 1 hour


    number_of_transactions = {}

    for i in df_shopid_complete.keys(): #taking each shopid
        
        temp_df = {} #temporary dict to hold the userid total transactions in each shopid within 1 hour instance
        
        for x in df_shopid_complete[i].keys(): #taking each userid in each shopid
            
            len_userid_unixtime = len(list(df_shopid_complete[i][x]['unix_time']))-1 #the last index in unix_time column
            
            while len_userid_unixtime > 1: #index 0 or 1 means the unix_time length is only 2 or less, dont bother with it since it must be not an order brushing
                #ans is the subtracted value of 1st and last value in unix time stamp of a userid
                ans = abs(list(df_shopid_complete[i][x]['unix_time'])[0] - list(df_shopid_complete[i][x]['unix_time'])[len_userid_unixtime])
                if ans > 3600 : #if the value is more than 1 hour
                    len_userid_unixtime -= 1 # decrement the last index

                elif ans <= 3600: #if the value is within 1 hour
                    temp_df[x] = len_userid_unixtime + 1 #len_userid_unixtime value is in index number (ordinal),
                    #add 1 to convert to actual length (sum)
                    break
          
        number_of_transactions[i] = temp_df #finally adds our temporary dictionary to our answer dictionary
    #now to clean this up

#there are dict inside this nested dict with empty value since that shopid didn't do any order brushing
#drop shopid with empty dict
    for i in list(number_of_transactions.keys()):
        if len(number_of_transactions[i]) == 0:
            del number_of_transactions[i]
    
    
    #make dict of sus shopid that has more than 1 sus userid connected by "&"
    shopid_suspect = list(number_of_transactions.keys()) #unique shopid
    shopid_userid_suspect_pair = {} #finished dict of shopid with '&' if 1 more userid detected in a shopid
    for i in shopid_suspect:
        sortit = sorted(list(number_of_transactions[i].keys()))
        shopid_userid_suspect_pair[i] = '&'.join(map(str, sortit)) #join userid with &, if userid only 1 transaction it will stay the same dont worry
        
    
    #now join suspected with non-suspected in a dict and later convert to dataframe
    #make original df
    orig_df = df

    all_pair = {}
    for i in orig_df['shopid']: #take every  shopid in our original df 
        #remember in our original df, the same shopid can appear multiple times
        if i in all_pair:
            all_pair[i] = all_pair[i]
            continue
        #then
        if i in shopid_userid_suspect_pair: #check if the shopid is inside our shopid_userid_suspect_pair dict
            all_pair[i] = shopid_userid_suspect_pair[i] #if yes then add that value
            continue #continue to the next value
        
        if i not in shopid_userid_suspect_pair: #if the value is not in the shopid_userid_suspect_pair dict
            all_pair[i] = str(0)

    #now convert to df
    suspect = pd.DataFrame(list(all_pair.items()),columns = ['shopid','userid'])
    return suspect #this will return df  complete form

In [27]:
suspect_order_brushing = order_brush(df) #call the function and put the dataframe result in a variable
suspect_order_brushing

,shopid,userid
0,93950878,0
1,156423439,0
2,173699291,0
3,63674025,0
4,127249066,0
...,...,...
18765,163337224,0
18766,772599,0
18767,50236030,0
18768,203587596,0


In [28]:
stop = timeit.default_timer()

print('Time: ', stop - start)  #find the total running time
#30 second running time

Time:  30.613837700000204


In [29]:
suspect_order_brushing.to_csv('submissions.csv', index=False) #save the answer in a csv